In [2]:
import pandas as pd

In [3]:
dfsa=pd.read_csv("C:/Users/ASUS/Downloads/Sales.csv")

In [4]:
dfsa

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2/20/2021,NaN,1216913,43,632,3,USD
62880,2243031,1,2/20/2021,2/24/2021,511229,0,98,4,EUR
62881,2243032,1,2/20/2021,2/23/2021,331277,0,1613,2,CAD
62882,2243032,2,2/20/2021,2/23/2021,331277,0,1717,2,CAD


In [5]:
dfsa.isnull().sum()

Order Number         0
Line Item            0
Order Date           0
Delivery Date    49719
CustomerKey          0
StoreKey             0
ProductKey           0
Quantity             0
Currency Code        0
dtype: int64

In [6]:
(dfsa.isnull().sum()/len(dfsa))*100

Order Number      0.000000
Line Item         0.000000
Order Date        0.000000
Delivery Date    79.064627
CustomerKey       0.000000
StoreKey          0.000000
ProductKey        0.000000
Quantity          0.000000
Currency Code     0.000000
dtype: float64

In [7]:
dfsa.dtypes

Order Number      int64
Line Item         int64
Order Date       object
Delivery Date    object
CustomerKey       int64
StoreKey          int64
ProductKey        int64
Quantity          int64
Currency Code    object
dtype: object

In [8]:
# Convert date columns to datetime format
dfsa['Order Date'] = pd.to_datetime(dfsa['Order Date'], format='%m/%d/%Y')

In [9]:
dfsa.drop(columns=['Delivery Date'], inplace=True)

In [10]:
dfsa

,Order Number,Line Item,Order Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,2016-01-01,265598,10,1304,1,CAD
1,366001,1,2016-01-01,1269051,0,1048,2,USD
2,366001,2,2016-01-01,1269051,0,2007,1,USD
3,366002,1,2016-01-01,266019,0,1106,7,CAD
4,366002,2,2016-01-01,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...
62879,2243030,1,2021-02-20,1216913,43,632,3,USD
62880,2243031,1,2021-02-20,511229,0,98,4,EUR
62881,2243032,1,2021-02-20,331277,0,1613,2,CAD
62882,2243032,2,2021-02-20,331277,0,1717,2,CAD


In [11]:
dfsa.dtypes

Order Number              int64
Line Item                 int64
Order Date       datetime64[ns]
CustomerKey               int64
StoreKey                  int64
ProductKey                int64
Quantity                  int64
Currency Code            object
dtype: object

In [12]:
dfsa.isnull().sum()

Order Number     0
Line Item        0
Order Date       0
CustomerKey      0
StoreKey         0
ProductKey       0
Quantity         0
Currency Code    0
dtype: int64

In [13]:
dfsa.duplicated().sum()

0

In [14]:
dfsa.drop_duplicates(inplace=True)

In [15]:
dfsa.tail()

,Order Number,Line Item,Order Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
62879,2243030,1,2021-02-20,1216913,43,632,3,USD
62880,2243031,1,2021-02-20,511229,0,98,4,EUR
62881,2243032,1,2021-02-20,331277,0,1613,2,CAD
62882,2243032,2,2021-02-20,331277,0,1717,2,CAD
62883,2243032,3,2021-02-20,331277,0,464,7,CAD


In [16]:
dfsa.to_csv("D:\CHITRA\DATASPARK\clean_sales.csv",index=False)

In [17]:
import pandas as pd
import pymysql

#CSV file into a DataFrame
sales_df = pd.read_csv("D:\CHITRA\DATASPARK\clean_sales.csv")

#Connect to the MySQL database
connection = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='2210',
    database='dataspark'
)

try:
    #Create the 'sales' table with foreign key constraints
    create_table_query = """
    CREATE TABLE IF NOT EXISTS sales (
        Order_Number INT,
        Line_Item INT,
        Order_Date DATE,
        CustomerKey INT,
        StoreKey INT,
        ProductKey INT,
        Quantity INT,
        Currency_Code VARCHAR(255),
        PRIMARY KEY (Order_Number, Line_Item),
        FOREIGN KEY (StoreKey) REFERENCES stores(StoreKey),
        FOREIGN KEY (ProductKey) REFERENCES products(ProductKey)
    );
    """
    
    with connection.cursor() as cursor:
        cursor.execute(create_table_query)
        connection.commit()
        print("Table 'sales' created successfully with foreign keys!")

    #Insert data from DataFrame into the 'sales' table
    insert_query = """
    INSERT INTO sales (Order_Number, Line_Item, Order_Date, CustomerKey, StoreKey, ProductKey, Quantity, Currency_Code)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    with connection.cursor() as cursor:
        for index, row in sales_df.iterrows():
            cursor.execute(insert_query, (
                row['Order Number'],
                row['Line Item'],
                row['Order Date'],
                row['CustomerKey'],
                row['StoreKey'],
                row['ProductKey'],
                row['Quantity'],
                row['Currency Code']
            ))
        connection.commit()
        print("Data inserted successfully into 'sales' table!")

finally:
    #Close the connection
    connection.close()


Table 'sales' created successfully with foreign keys!
Data inserted successfully into 'sales' table!
